# Hedge Fund X
This notebook solve the Hedge Fund X's competition challenge: Financial Modeling challenge.
This one is used to evaluate model, tuning param to find most sutable model.
To discover data set, check the other Notebook.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn import metrics
from xgboost import XGBClassifier

import datetime

In [6]:
df = pd.read_csv("../input/hedge_fund_x/train.csv")
df_test = pd.read_csv("../input/hedge_fund_x/test.csv")
df.head(5)

,data_id,period,c1,c2,c3,c4,c5,c6,c7,c8,...,c80,c81,c82,c83,c84,c85,c86,c87,c88,target
0,2,train1,0.655570,-0.000022,-0.000539,-0.001075,0.0,0.0,0.213390,0.0,...,-0.023358,-0.017041,0.0,0.060697,0.0,0.0,0.000000,-0.000202,-0.140220,1
1,3,train1,1.646430,-0.000292,-0.008367,0.009497,0.0,0.0,0.000000,0.0,...,-0.059429,-0.009109,0.0,0.021645,0.0,0.0,0.000000,-0.004382,0.455767,0
2,5,train1,-0.743010,0.004642,-0.000647,-0.003290,0.0,0.0,0.000000,0.0,...,0.001796,-0.000104,0.0,-0.024718,0.0,0.0,0.219566,0.072711,1.155580,0
3,7,train1,0.029770,-0.006343,-0.000635,-0.002516,0.0,0.0,0.160313,0.0,...,-0.005501,0.045308,0.0,-0.148852,0.0,0.0,0.000000,-0.101181,-0.954553,0
4,10,train1,-0.660243,0.012591,-0.002098,-0.022264,0.0,0.0,0.000000,0.0,...,0.029034,-0.005847,0.0,-0.007073,0.0,0.0,0.000000,-0.004842,0.436002,0


In [7]:
# We drop column period to make it same as test dataset
train1_df = df[df.period == 'train1'].drop(['period'], axis=1)

## Training
### Model definition
Define multiple models to test
_NOTE:_ Work in progress
_TODO:_ review all pipeline

In [8]:
clf_xgb = XGBClassifier(max_depth=7, n_estimators=150, reg_lambda=10)

### Parameter tuning

### Evaluate model
compare performance of model

In [126]:
def print_val_score(scores, label):
    print "{}: {:.2f} (+/- {:.2f})".format(
        label,
        scores[label].mean(), 
        scores[label].std())
def evaluate_model(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_train = train_df[selected_cols].values
    y_train = train_df['target'].values

    scoring={'acc':'accuracy', 'log_loss':'neg_log_loss'}
    scores = cross_validate(estimator=est, X=X_train, y=y_train, cv=10, scoring=scoring, 
                            return_train_score=True)
    print_val_score(scores, 'train_acc')
    print_val_score(scores, 'test_acc')
    print_val_score(scores, 'train_log_loss')
    print_val_score(scores, 'test_log_loss')

### Baseline performance
We treat this as a baseline for our optimization, if we get over this, we are getting better

In [33]:
evaluate_model(clf_xgb, train1_df)

train_acc: 0.92 (+/- 0.00)
test_acc: 0.82 (+/- 0.10)
train_log_loss: -0.33 (+/- 0.01)
test_log_loss: -0.44 (+/- 0.09)


### Use model to predict next period

In [76]:
selected_cols = [col for col in df.columns if col not in ['period', 'target']]
train1_df = df[df.period == 'train1'].drop(['period'], axis=1)
X_train = train1_df[selected_cols].values
y_train = train1_df['target'].values

clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=10,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [116]:
def evaluate_period(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_test = train_df[selected_cols].values
    y_test = train_df['target'].values
    y_pred = est.predict(X_test)
    y_pred_proba = clf_xgb.predict_proba(X_test)[:,1]
    return y_pred, y_pred_proba, y_test

In [117]:
def print_pred_score(y_pred, y_pred_proba, y_test):
    print "Accuracy scrore: {}".format(metrics.accuracy_score(y_test, y_pred))
    print "Log loss scrore: {}".format(metrics.log_loss(y_test, y_pred_proba))

In [80]:
train2_df = df[df.period == 'train2'].drop(['period'], axis=1)

In [81]:
print_pred_score(*evaluate_period(clf_xgb, train1_df))
print_pred_score(*evaluate_period(clf_xgb, train2_df))

Accuracy scrore: 0.9168
Log loss scrore: 0.339864600846
Accuracy scrore: 0.498525
Log loss scrore: 0.751101043463


Naive Xgboost doesn't do well on next period. Look like the schematics of different period is different. Let try other

In [82]:
train3_df = df[df.period == 'train3'].drop(['period'], axis=1)
train4_df = df[df.period == 'train4'].drop(['period'], axis=1)
train5_df = df[df.period == 'train5'].drop(['period'], axis=1)

In [83]:
print_pred_score(*evaluate_period(train3_df))
print_pred_score(*evaluate_period(train4_df))
print_pred_score(*evaluate_period(train5_df))

Accuracy scrore: 0.48865
Log loss scrore: 0.77315410604
Accuracy scrore: 0.50055
Log loss scrore: 0.759808796312
Accuracy scrore: 0.50835
Log loss scrore: 0.750535000322


The accuracy score is 0.5, which means we are as worse as random!

## Remove noise
There are some column that has so much noise, going to train with those collumn removed

In [114]:
zero_count = (train1_df==0.0).sum(axis=0).sort_values(ascending=False)
has_zero_ratio = zero_count[zero_count > 0].div(train1_df.shape[0])
excluded_cols = has_zero_ratio.index
print excluded_cols.values

['c8' 'c37' 'c34' 'c45' 'c23' 'c18' 'c6' 'c62' 'c28' 'c85' 'c46' 'c60'
 'c55' 'c65' 'c5' 'c32' 'c41' 'c54' 'c86' 'c70' 'c63' 'c82' 'c7' 'c19'
 'c58' 'c52' 'c17' 'c10' 'c13' 'c84' 'c75' 'c42' 'target']


In [128]:
train1_df = df[df.period == 'train1'].drop(['period'], axis=1)
selected_cols = [col for col in train1_df.columns if col not in excluded_cols]
X_train = train1_df[selected_cols].values
y_train = train1_df['target'].values

clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train1_df, excluded_cols)

train_acc: 0.93 (+/- 0.00)
test_acc: 0.83 (+/- 0.10)
train_log_loss: -0.30 (+/- 0.01)
test_log_loss: -0.42 (+/- 0.10)


In [127]:
print_pred_score(*evaluate_period(train1_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train2_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train3_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train4_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train5_df, clf_xgb, excluded_cols))

Accuracy scrore: 0.927875
Log loss scrore: 0.298220415532
Accuracy scrore: 0.4979
Log loss scrore: 0.761795944967
Accuracy scrore: 0.48895
Log loss scrore: 0.780703311869
Accuracy scrore: 0.49895
Log loss scrore: 0.770825451828
Accuracy scrore: 0.512425
Log loss scrore: 0.760564929533


Remove missing data increase performance of tree (val also increase, but make no sense to validation)

## Config parameter

In [134]:
train1_df = df[df.period == 'train1'].drop(['period'], axis=1)
selected_cols = [col for col in train1_df.columns if col not in excluded_cols]
X_train = train1_df[selected_cols].values
y_train = train1_df['target'].values

clf_xgb = XGBClassifier(max_depth=7, n_estimators=150, reg_lambda=100)
clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train1_df, excluded_cols)

train_acc: 0.90 (+/- 0.00)
test_acc: 0.84 (+/- 0.03)
train_log_loss: -0.37 (+/- 0.00)
test_log_loss: -0.44 (+/- 0.02)


In [135]:
print_pred_score(*evaluate_period(train2_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train3_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train4_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train5_df, clf_xgb, excluded_cols))

Accuracy scrore: 0.487325
Log loss scrore: 0.738585041917
Accuracy scrore: 0.482875
Log loss scrore: 0.74065795263
Accuracy scrore: 0.486175
Log loss scrore: 0.745111142795
Accuracy scrore: 0.502975
Log loss scrore: 0.728499262134


## Evaluate next period by group of previous period
Try to see if data on a **group** of period can help to increase prediction accuracy on other period

In [141]:
train_df = pd.concat([train1_df, train2_df, train3_df, train4_df])
X_train = train_df[selected_cols].values
y_train = train_df['target'].values
print train_df.shape

(160000, 90)


In [142]:
clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train_df, excluded_cols)

train_acc: 0.82 (+/- 0.00)
test_acc: 0.72 (+/- 0.04)
train_log_loss: -0.53 (+/- 0.00)
test_log_loss: -0.59 (+/- 0.02)


In [143]:
print_pred_score(*evaluate_period(train1_df, clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train5_df, clf_xgb, excluded_cols))

Accuracy scrore: 0.810325
Log loss scrore: 0.548924755372
Accuracy scrore: 0.491525
Log loss scrore: 0.716300661858


Group of training data on 4 periods doesn't increase accuracy of the 5th period. So:
1. There is bug in our model (I config Xgboost wrong)
2. Model is not have enough capability to capture feature (I doubt this)
3. The data between 2 periods is nonsense

### Try to train half-half
Use 75% of a period to predict 25% of that period

In [144]:
train_df = pd.concat([train1_df[:30000], train2_df[:30000], train3_df[:30000], train4_df[:30000]])
X_train = train_df[selected_cols].values
y_train = train_df['target'].values
print train_df.shape

(120000, 90)


In [145]:
clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train_df, excluded_cols)

train_acc: 0.82 (+/- 0.00)
test_acc: 0.73 (+/- 0.02)
train_log_loss: -0.53 (+/- 0.00)
test_log_loss: -0.59 (+/- 0.02)


In [146]:
print_pred_score(*evaluate_period(train1_df[30000:], clf_xgb, excluded_cols))
print_pred_score(*evaluate_period(train5_df[30000:], clf_xgb, excluded_cols))

Accuracy scrore: 0.7547
Log loss scrore: 0.584702878413
Accuracy scrore: 0.5091
Log loss scrore: 0.708663111234


There is tiny improvement, but not enough

### Try on single period to train half-half

In [147]:
train_df = train1_df[:30000]
X_train = train_df[selected_cols].values
y_train = train_df['target'].values
print train_df.shape

(30000, 90)


In [148]:
clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train_df, excluded_cols)

train_acc: 0.91 (+/- 0.00)
test_acc: 0.84 (+/- 0.01)
train_log_loss: -0.37 (+/- 0.00)
test_log_loss: -0.45 (+/- 0.01)


In [149]:
print_pred_score(*evaluate_period(train1_df[30000:], clf_xgb, excluded_cols))

Accuracy scrore: 0.8348
Log loss scrore: 0.447209817151


Yes! Data within same period get a very good result. Try again with period 2

In [150]:
train_df = train2_df[:30000]
X_train = train_df[selected_cols].values
y_train = train_df['target'].values
print train_df.shape

(30000, 90)


In [151]:
clf_xgb.fit(X_train, y_train)
evaluate_model(clf_xgb, train_df, excluded_cols)

train_acc: 0.90 (+/- 0.00)
test_acc: 0.81 (+/- 0.01)
train_log_loss: -0.40 (+/- 0.00)
test_log_loss: -0.48 (+/- 0.00)


In [152]:
print_pred_score(*evaluate_period(train2_df[30000:], clf_xgb, excluded_cols))

Accuracy scrore: 0.8099
Log loss scrore: 0.486093644936


So look like feature has different meaning between periods. So we can try to detect which period it's first, then predict the target.

### Predict period

In [168]:
clf_xgb_period = XGBClassifier(max_depth=7, n_estimators=150, reg_lambda=100, 
                               objective='multi:softmax')

In [220]:
train_df = pd.concat([df[x*40000: x*40000 + 5000] for x in range(0,7)])
print train_df.shape

(35000, 91)


In [221]:
def evaluate_model(est, train_df, excluded_cols = ['period', 'target']):
    selected_cols = [col for col in train_df.columns if col not in excluded_cols]
    X_train = train_df[selected_cols].values
    y_train = train_df['period'].values
#     le = LabelEncoder()
#     le.fit(y_train)
#     y_train = le.transform(y_train)

    scoring={'acc':'accuracy', 'log_loss':'neg_log_loss'}
    scores = cross_validate(estimator=est, X=X_train, y=y_train, cv=10, scoring=scoring)
    print_val_score(scores, 'train_acc')
    print_val_score(scores, 'test_acc')
    print_val_score(scores, 'train_log_loss')
    print_val_score(scores, 'test_log_loss')

In [222]:
excluded_cols_period = [x for x in excluded_cols]
excluded_cols_period.append('period')

In [223]:
evaluate_model(clf_xgb_period, train_df, excluded_cols_period)

train_acc: 1.00 (+/- 0.00)
test_acc: 1.00 (+/- 0.00)
train_log_loss: -0.00 (+/- 0.00)
test_log_loss: -0.00 (+/- 0.00)


### Predict period for data

In [ ]:
oh = OneHotEncoder()
oh_period = oh.fit_transform(df['period'].values)
oh_df = pd.concat([df, oh_period], axis=1)
print oh_df.head(5)

### Plot learning curves
Train model multiple times on different training set size to inspect overfit or underfit

In [2]:
from sklearn.model_selection import learning_curve

def plot_learning_curve(X_train, y_train, est, title):
    train_sizes, train_scores, test_scores = learning_curve(estimator=est,
                       X=X_train,
                       y=y_train,
                       train_sizes=np.linspace(0.1, 1.0, 10),
                       cv=10,
                       n_jobs=1)

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.plot(train_sizes, train_mean,
             color='blue', marker='o',
             markersize=5, label='training accuracy')

    plt.fill_between(train_sizes,
                     train_mean + train_std,
                     train_mean - train_std,
                     alpha=0.15, color='blue')

    plt.plot(train_sizes, test_mean,
             color='green', linestyle='--',
             marker='s', markersize=5,
             label='validation accuracy')

    plt.fill_between(train_sizes,
                     test_mean + test_std,
                     test_mean - test_std,
                     alpha=0.15, color='green')

    plt.grid()
    plt.title(title)
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.2, 1.0])
    plt.tight_layout()
    plt.savefig('./figures/learning_curve_xgboost_{:%Y%m%d-%H%M}.png'.format(datetime.datetime.now()), dpi=300)
    plt.show()

In [30]:
plot_learning_curve(X_train, y_train, clf_xgb, 'xgb')

## Prediction
Prepare train & test data for predition model

In [130]:
# Train all data, to train a subset
# selected_cols = [col for col in train1_df.columns if col not in ['period', 'target']]
selected_cols = [col for col in train1_df.columns if col not in excluded_cols]
X_train = df[selected_cols].values
y_train = df['target'].values
X_test = df_test[selected_cols].values
print 'Train data shape: {}'.format(X_train.shape)
print 'Train data shape: {}'.format(X_test.shape)

Train data shape: (560000, 57)
Train data shape: (361500, 57)


Fit all training data (above code is model with cross validation) and make prediction

In [131]:
chosen_est = clf_xgb
chosen_est.fit(X_train, y_train)
predictions = chosen_est.predict_proba(X_test)[:,1]
submission = pd.DataFrame({'data_id': df_test['data_id'],'target': predictions})
submission.to_csv("submit_{:%Y%m%d-%H%M}.csv".format(datetime.datetime.now()), index=False)